<a href="https://colab.research.google.com/github/Inyrkz/Politician-db/blob/master/Politician.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Inyrkz/Politician-db.git

Cloning into 'Politician-db'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [3]:
import pandas as pd
METADATA = pd.read_csv('/content/Politician-db/Politician DS.csv')
METADATA.head(3)

,S/N,Name,Position,Political Party,Elected
0,1,Okezie Ikpeazu,Current Governor of Abia State,PDP,2019.0
1,2,Ahmadu Umaru Fintiri,Current Governor of Adamawa State,PDP,2019.0
2,3,Udom Gabriel Emmanuel,Current Governor of Akwa-Ibom State,PDP,2019.0


In [5]:
#Joining the other columns into one 
METADATA["General"] = METADATA["Position"] + " " + METADATA["Political Party"] 
# + " " +  METADATA["Elected"]
METADATA["General"].head()

0         Current Governor of Abia State PDP
1      Current Governor of Adamawa State PDP
2    Current Governor of Akwa-Ibom State PDP
3     Current Governor of Anambra State APGA
4          Deputy Governor of Abia State PDP
Name: General, dtype: object

In [0]:
#Importing Libraries
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer object. Remove all english stopwords
TFIDF = TfidVectorizer(stop_words = 'english')

